### Glowpick
- 뷰티, 코스메틱 정보제공 웹/앱 서비스 
- https://www.glowpick.com/

In [1]:
import pandas as pd
import scrapy
import requests
from scrapy.http import TextResponse
from fake_useragent import UserAgent

In [2]:
# 1. 프로젝트 생성

In [3]:
!rm -rf glowpick
!scrapy startproject glowpick

New Scrapy project 'glowpick', using template directory '/Users/jhkim/opt/miniconda3/lib/python3.7/site-packages/scrapy/templates/project', created in:
    /Users/jhkim/Documents/dev/TIL_/glowpick

You can start your first spider with:
    cd glowpick
    scrapy genspider example example.com


In [4]:
!tree glowpick

glowpick
├── glowpick
│   ├── __init__.py
│   ├── __pycache__
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── __pycache__
└── scrapy.cfg

4 directories, 7 files


In [5]:
# 2. items.py 코드 작성

In [6]:
%%writefile glowpick/glowpick/items.py
import scrapy

class GlowpickItem(scrapy.Item):
    brend = scrapy.Field()
    name = scrapy.Field()
    score_rating = scrapy.Field()
    pirce = scrapy.Field()
    reviews = scrapy.Field()

Overwriting glowpick/glowpick/items.py


In [7]:
# 3. xpath 찾기 : 링크, 상품 데이터

In [8]:
# 링크

In [9]:
url = "https://www.glowpick.com/beauty/ranking?id=14&level=2"
# headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
headers = {"User-Agent": UserAgent().chrome}
req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://www.glowpick.com/beauty/ranking?id=14&level=2>

In [11]:
link = response.xpath('//*[@id="gp-list"]/div/section[2]/ul/li/meta/@content').extract()# //*[@id="gp-list"]/div/section[2]
len(link)

20

In [12]:
link = link[0] # //*[@id="gp-list"]/div/section[2]/ul/li[1]/div/div/meta

In [13]:
req = requests.get(link)
response = TextResponse(req.url, body = req.text, encoding='utf-8')

In [14]:
brend = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/p/text()').extract()[0].strip()
name = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/h1/span/text()').extract()[0].strip()
score_rating = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/div[2]/div[2]/span[1]/text()')[0].extract()
price = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/div[2]/div[1]/span/text()')[0].extract()
reviews = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/div[2]/div[2]/span[2]/text()').extract()[0].strip()
brend ,name, score_rating, price, reviews

('바비브라운 (BOBBI BROWN)', '럭스 립 칼라', '4.38', '44,000', '(2,545)')

In [15]:
# 4. spider.py 코드 작성 : fake user agent 설정 //*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/p

In [16]:
#!pip install scrapy-fake-useragent

In [29]:
%%writefile glowpick/glowpick/spiders/spider.py
import scrapy
from glowpick.items import GlowpickItem

class GlowpickSpider(scrapy.Spider):
    name = "Glowpick"
    start_url = ["https://www.glowpick.com/beauty/ranking?id=14&level=2"]
    custom_settings = {
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadmiddlewares.useragent.UserAgentMiddleware": None,
            "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
        }
    }
    
    def parse(self, response):
        links = response.xpath('//*[@id="gp-list"]/div/section[2]/ul/li/meta/@content').extract() 
        links = [link for link in links]
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
        
        
        
    def parse_content(self, response):
        item = GlowpicItem()
        brend = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/p/text()').extract()[0].strip()
        name = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/h1/span/text()').extract()[0].strip()
        score_rating = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/div[2]/div[2]/span[1]/text()')[0].extract()
        price = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/div[2]/div[1]/span/text()')[0].extract()
        reviews = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/div[2]/div[2]/span[2]/text()').extract()[0].strip()
        
        item["brend"] = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/p/text()').extract()[0].strip()
        item["name"] = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/h1/span/text()').extract()[0].strip()
        item["score_rating"] = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/div[2]/div[2]/span[1]/text()')[0].extract()
        item["pirce"] = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/div[2]/div[1]/span/text()')[0].extract()
        item["reviews"] = response.xpath('//*[@id="gp-default-main"]/section/div/ul[1]/li[2]/section[1]/div[2]/div[2]/span[2]/text()').extract()[0].strip()
        yield item

Overwriting glowpick/glowpick/spiders/spider.py


In [30]:
# 5. 스크래피 실행시 아규먼트 설정

In [31]:
# 6. 스크레피 실행

In [32]:
%%writefile run.sh
cd glowpick
scrapy crawl Glowpick -o glowpick_test_7.csv

Overwriting run.sh


In [33]:
!source run.sh

2020-07-12 20:20:56 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: glowpick)
2020-07-12 20:20:56 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.4 (default, Aug 13 2019, 15:17:50) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Darwin-19.5.0-x86_64-i386-64bit
2020-07-12 20:20:56 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'glowpick', 'FEED_FORMAT': 'csv', 'FEED_URI': 'glowpick_test_7.csv', 'NEWSPIDER_MODULE': 'glowpick.spiders', 'SPIDER_MODULES': ['glowpick.spiders']}
2020-07-12 20:20:56 [scrapy.extensions.telnet] INFO: Telnet Password: ce33003d74afc9ea
2020-07-12 20:20:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logs

In [26]:
pd.read_csv("glowpick/glowpick_test_7.csv")

EmptyDataError: No columns to parse from file

In [27]:
!head -n 25 glowpick/glowpick/settings.py | tail -n 5

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32


In [28]:
# !sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' glowpick/glowpick/settings.py
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' glowpick/glowpick/settings.py

sed: 1: "glowpick/glowpick/setti ...": extra characters at the end of g command
